In [13]:
# read in the csv data containing the barcodes
from sierra_ils_utils import QueryBuilder
import csv

with open('./current_collection-f1db491.csv', 'r') as f:
    reader = csv.reader(f)

    columns = next(reader)  # the first row is the column names (in this case just one)
    rows = [row[0] for row in reader]  # the first column is the barcode

q1 = QueryBuilder()  # create a query builder object using our barcodes
q1.start_query(record_type='item', field_tag='b') \
    .add_expression(op='in', operands=rows) \
    .end_query() \
    
q2 = QueryBuilder()  # create a second query builder object
q2.start_query(record_type='item', id=88) \
    .add_expression(op='regex_exact', operands='-') \
    .end_query()

try:
  complex_query = q1 + q2  # the `+` here is a logical AND that stitches the two together ... 
except:
   TypeError  # sometime soon, this won't throw an error!

# resulting in the following:
# complex_query = """\
# {
#   "queries": [
#     {
#       "target": {
#         "record": {
#           "type": "item"
#         },
#         "field": {
#           "tag": "b"
#         }
#       },
#       "expr": [
#         {
#           "op": "in",
#           "operands": [
#             "a000005935549",
#             "a000041304478",
#             "a000041304486",
#             "a000048891352",
#             "a000048891360",
#             "a000048891402",
#             "a000062669777",
#             "a000070823168",
#             "a000070823176",
#             "a000076582628",
#             "a000076582636",
#             "a000076582644",
#             "a000076582651"
#           ]
#         }
#       ]
#     },
#     "and",
#     {
#       "target": {
#         "record": {
#           "type": "item"
#         },
#         "id": 88
#       },
#       "expr": [
#         {
#           "op": "regex_exact",
#           "operands": [
#             "-"
#           ]
#         }
#       ]
#     }
#   ]
# }"""



{
    "raw_response": "<Response [200 200]>",
    "response_model_name": "QueryResultSet",
    "data": {
        "total": 6,
        "start": 0,
        "entries": [
            {
                "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/7885514"
            },
            {
                "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/8564848"
            },
            {
                "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/8564849"
            },
            {
                "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/10237813"
            },
            {
                "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/11769596"
            },
            {
                "link": "https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/11769597"
            }
        ]
    }
}

In [ ]:
from sierra_ils_utils import SierraAPI
import json

# Load configuration
with open('./.config.json') as f:
    config = json.load(f)

# Initialize SierraRESTAPI with DEBUG logging level
sierra_api = SierraAPI(
    sierra_api_base_url=config.get('sierra_api_base_url'),
    sierra_api_key=config.get('sierra_api_key'),
    sierra_api_secret=config.get('sierra_api_secret')
)

results = sierra_api.post(
    'items/query', 
    params={
        'limit': 2000,
        'offset': 0
    },
    json_body=complex_query  # the complex query here is a list of items (from their barcodes)
                             # ... AND items have status `-`
)

results

In [5]:
q = QueryBuilder()  # create a query builder object using our barcodes

q.start_query(record_type='item', field_tag='b') \
    .add_expression(op='in', operands=rows) \
    .end_query() \
    .add_logical_operator('and') \
    .start_query(record_type='item')

q.start_query(record_type='item', id=88) \
    .add_expression(op='regex_exact', operands='-') \
    .end_query()

{
  "target": {
    "record": {
      "type": "item"
    },
    "id": 88
  },
  "expr": [
    {
      "op": "regex_exact",
      "operands": [
        "-"
      ]
    }
  ]
}

In [1]:
import json
import logging
from sierra_ils_utils import SierraAPI, RecordDateRange, endpoints, QueryBuilder

# Configure the root logger
logging.basicConfig(level=logging.DEBUG)

# Load configuration
with open('./.config.json') as f:
    config = json.load(f)

# Create and configure the logger for the current module
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Optionally, set the logging level for the sierra_ils_utils module
logging.getLogger('sierra_ils_utils').setLevel(logging.DEBUG)

# Initialize SierraRESTAPI with DEBUG logging level
sierra_api = SierraAPI(
    sierra_api_base_url=config.get('sierra_api_base_url'),
    sierra_api_key=config.get('sierra_api_key'),
    sierra_api_secret=config.get('sierra_api_secret'),
    log_level=logging.DEBUG
)

DEBUG:sierra_ils_utils.sierra_ils_utils:INIT {'base_url': 'https://classic.cincinnatilibrary.org/iii/sierra-api/v6/', 'api_key': '1ebmZG6N********************', 'request_count': 0, 'expires_at': 0, 'session_headers': {'accept': 'application/json', 'Authorization': ''}, 'endpoints': {'GET': {'bibs/': {'responses': {200: <class 'sierra_ils_utils.sierra_api_v6_endpoints.BibResultSet'>, 400: <class 'sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode'>, 404: <class 'sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode'>}, 'response_model': <class 'sierra_ils_utils.sierra_api_v6_endpoints.BibResultSet'>}, 'bibs/{id}': {'responses': {200: <class 'sierra_ils_utils.sierra_api_v6_endpoints.Bib'>, 400: <class 'sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode'>, 404: <class 'sierra_ils_utils.sierra_api_v6_endpoints.ErrorCode'>}, 'response_model': <class 'sierra_ils_utils.sierra_api_v6_endpoints.Bib'>}, 'info/token': {'responses': {200: <class 'sierra_ils_utils.sierra_api_v6_endpoints.TokenInfo'>},

In [2]:
# we have a list of 100 item barcodes in a .csv file ... 
# ... we're going to load them into a list

import csv

with open('./current_collection-a9be2bf.csv') as f:
    reader = csv.reader(f)             # use the csv reader to read the file
    columns = next(reader)             # first "row" is the column names
    rows = [row[0] for row in reader]  # produce a list of barcodes

print(
    columns,   # column names
    rows[:3],  # first 3 barcodes
    len(rows), # number of barcodes
    sep='\n'
)

# For now, lets's just deal with the first 20 barcodes
rows = rows[:20]

print(
    "---",
    columns,    # column names
    rows[:3],   # first 3 barcodes
    len(rows),  # number of barcodes
    sep='\n'
)

['barcode']
['0923112556017', 'A000074528052', '8264518556018']
100
---
['barcode']
['0923112556017', 'A000074528052', '8264518556018']
20


In [3]:
# sierra-ils-utils also has a (simple) query builder

# https://documentation.iii.com/sierrahelp/#sgil/sgil_lists_json.html
# > You can view and define Create Lists queries using JavaScript Object 
# > Notation (JSON)

query = QueryBuilder()  # instatiate the class

# the query builder allows for "method chaining": 
# ... below, we start a query with a target type of "item", and varfield tag "b"
# ... followed by the barcodes, "b" in the list of operands
# e.g. item records having a matching barcodes to ones in our list
query = query.start_query(record_type='item', field_tag='b') \
    .add_expression(op='in', operands=rows) \
    .end_query()  # end the query

In [4]:
query

{
  "target": {
    "record": {
      "type": "item"
    },
    "field": {
      "tag": "b"
    }
  },
  "expr": [
    {
      "op": "in",
      "operands": [
        "0923112556017",
        "A000074528052",
        "8264518556018",
        "A000068621988",
        "A000058040528",
        "1020643846014",
        "1062395-2001",
        "0995811352013",
        "A000054341995",
        "8100145849014",
        "A000023623580",
        "A000075775231",
        "A000074880321",
        "0937904152025",
        "A000072564059",
        "A000025464017",
        "0900611452138",
        "A000064695739",
        "A000011346970",
        "1023446846025"
      ]
    }
  ]
}

In [5]:
# send the query!
response = sierra_api.post(
    'items/query',
    params={
        'offset': 0,
        'limit': 20,
    },
    json_body=query.json()
)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): classic.cincinnatilibrary.org:443
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "POST /iii/sierra-api/v6/token HTTP/1.1" 200 230
INFO:sierra_ils_utils.sierra_ils_utils:Authorization Success. response.json.get('expires_in'): 3600
INFO:sierra_ils_utils.sierra_ils_utils:Sierra session authenticated
DEBUG:urllib3.connectionpool:https://classic.cincinnatilibrary.org:443 "GET /iii/sierra-api/v6/info/token HTTP/1.1" 200 3584
INFO:sierra_ils_utils.sierra_ils_utils:Sierra response status code                  : 200
INFO:sierra_ils_utils.sierra_ils_utils:Sierra 'expiresIn'                           : 3599
INFO:sierra_ils_utils.sierra_ils_utils:session expires at (UNIX Epoch)              : 1701561655.4550316
INFO:sierra_ils_utils.sierra_ils_utils:seconds left                                 : 3539.9450726509094
INFO:sierra_ils_utils.sierra_ils_utils:request url                                  : https://clas

In [8]:
response.data

QueryResultSet(total=20, start=0, entries=[QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/1051453'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/2020707'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/2030791'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/2268916'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/2374433'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/3191696'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/3791673'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/3831990'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/3975242'), QueryEntry(link='https://classic.cincinnatilibrary.org/iii/sierra-api/v6/items/4782511'), QueryEntry(link='https://classic.cincinnatilibrary.org/ii

In [5]:
import json

class SierraQueryBuilder:
    def __init__(self):
        self.queries = []
        self.current_query = None
        self.last_was_operator = False

    def start_query(self, record_type, field_tag):
        if self.current_query is not None:
            raise ValueError("Previous query not ended. Use end_query to finish.")
        if self.last_was_operator:
            self.last_was_operator = False
        self.current_query = {"target": {"record_type": record_type, "field_tag": field_tag}, "expr": []}
        return self

    def add_expression(self, op, operands):
        if self.current_query is None:
            raise ValueError("No active query. Use start_query to begin.")
        if not isinstance(operands, list):
            operands = [operands]
        expression = {"op": op, "operands": operands}
        # The line below has been commented out to remove the restriction
        # if self.current_query["expr"] and isinstance(self.current_query["expr"][-1], str):
        #     raise ValueError("Must add logical operator before adding another expression to the same query.")
        self.current_query["expr"].append(expression)
        return self

    def add_logical_operator(self, operator):
        if operator not in ['and', 'or']:
            raise ValueError("Operator must be 'and' or 'or'.")
        if self.current_query and not isinstance(self.current_query["expr"][-1], str):
            self.current_query["expr"].append(operator)
        elif not self.queries or self.last_was_operator:
            raise ValueError("Cannot add a logical operator at this point.")
        else:
            self.queries.append(operator)
            self.last_was_operator = True
        return self

    def end_query(self):
        if self.current_query is None:
            raise ValueError("No active query to end.")
        if isinstance(self.current_query["expr"][-1], str):
            raise ValueError("Cannot end query after a logical operator. Add another expression.")
        if len(self.current_query["expr"]) == 1:
            self.current_query["expr"] = self.current_query["expr"][0]
        self.queries.append(self.current_query)
        self.current_query = None
        return self

    def build(self):
        if self.current_query is not None:
            raise ValueError("Query not ended. Use end_query to finish.")
        if self.last_was_operator:
            raise ValueError("Query structure ended with a logical operator.")
        return {"queries": self.queries}

    def json(self):
        return self.build()

    def __str__(self):
        return json.dumps(self.build(), indent=2)
    
    def __repr__(self) -> str:
        return self.__str__()


In [6]:
# Example 1: A simple query with a single, simple expression
# A simple query for Bibliographic title equal to 'birds of america'

# {
#   "target": {
#     "record": {"type": "bib"},
#     "field": {"tag": "t"}
#   },
#   "expr": {
#     "op": "equals",
#     "operands": ["birds of america"]
#   }
# }

query = SierraQueryBuilder()
query.start_query(record_type='bib', field_tag='t') \
    .add_expression(op='equals', operands=['birds of america']) \
    .end_query()

print(query)

{
  "queries": [
    {
      "target": {
        "record_type": "bib",
        "field_tag": "t"
      },
      "expr": {
        "op": "equals",
        "operands": [
          "birds of america"
        ]
      }
    }
  ]
}


In [7]:
query.json()

{'queries': [{'target': {'record_type': 'bib', 'field_tag': 't'},
   'expr': {'op': 'equals', 'operands': ['birds of america']}}]}

In [8]:
# Example 2: A compound query consisting of two simple queries
# A compound query for Bibliographic title equal to 'strangers on a train' and Bibliographic author has 'highsmith'

# {
#   "queries": [
#     {
#       "target": {
#         "record": {"type": "bib"},
#         "field": {"tag": "t"}
#       },
#       "expr": {
#         "op": "equals",
#         "operands": ["strangers on a train"]
#       }
#     },
#     "and",
#     {
#       "target": {
#         "record": {"type": "bib"},
#         "field": {"tag": "a"}
#       },
#       "expr": {
#         "op": "has",
#         "operands": ["highsmith"]
#       }
#     }
#   ]
# }

query = SierraQueryBuilder()
query.start_query(record_type='bib', field_tag='t') \
    .add_expression(op='equals', operands=['strangers on a train']) \
    .end_query() \
    .add_logical_operator('and') \
    .start_query(record_type='bib', field_tag='a') \
    .add_expression(op='has', operands=['highsmith']) \
    .end_query()

print(query)

{
  "queries": [
    {
      "target": {
        "record_type": "bib",
        "field_tag": "t"
      },
      "expr": {
        "op": "equals",
        "operands": [
          "strangers on a train"
        ]
      }
    },
    "and",
    {
      "target": {
        "record_type": "bib",
        "field_tag": "a"
      },
      "expr": {
        "op": "has",
        "operands": [
          "highsmith"
        ]
      }
    }
  ]
}


In [9]:
# Example 3: A simple query with a compound expression
# A simple query for Bibliographic title equal to 'moby dick' and has 'whale'

# {
#   "target": {
#     "record": {"type": "bib"},
#     "field": {"tag": "t"}
#   },
#   "expr": [
#     {
#       "op": "equals",
#       "operands": ["moby dick"]
#     },
#     "and",
#     {
#       "op": "has",
#       "operands": ["whale"]
#     }
#   ]
# }

query = SierraQueryBuilder()
query.start_query(record_type='bib', field_tag='t') \
    .add_expression(op='equals', operands=['moby dick']) \
    .add_logical_operator('and') \
    .add_expression(op='has', operands=['whale']) \
    .end_query()

print(query)

{
  "queries": [
    {
      "target": {
        "record_type": "bib",
        "field_tag": "t"
      },
      "expr": [
        {
          "op": "equals",
          "operands": [
            "moby dick"
          ]
        },
        "and",
        {
          "op": "has",
          "operands": [
            "whale"
          ]
        }
      ]
    }
  ]
}


In [10]:
# TODO: WHA?!

# Example 4: A soft-linked record type
# A query for Registered Session Summary starts with "gardening lecture"

# {
#   "target": {
#     "record": {
#       "type": "section",
#       "relationType": "soft",
#       "relationTag": "3"
#     },
#     "field": {"tag": "s"}
#   },
#   "expr": {
#     "op": "starts_with",
#     "operands": ["gardening lecture"]
#   }
# }

# query = SierraQueryBuilder()
# query.start_query(record_type='section', field_tag='s') \
#     .add_expression(op='equals', operands=['moby dick']) \
#     .add_logical_operator('and') \
#     .add_expression(op='has', operands=['whale']) \
#     .end_query()

# print(query)

In [11]:
# TODO: What have we gotten ourselves into

# Example 5: A compound query illustrating syntax for variable-length, fixed-length, and special fields
# A compound query for Bibliographic title has 'mockingbird' and Bibliographic MARC Tag 24501|c has "lee" and Bibliographic material type equal to 'a' and Bibliographic country equal to 'xxu' and Order paid date equals yesterday

# In this query, there are expressions targeting the following field types:

#     a variable-length field using a non-MARC field tag (title / field tag "t")
#     a variable-length field using a MARC field tag with indicators and a subfield (MARC 24501|c)
#     a fixed-length field (material type / ff #30)
#     a special field (MARC 008 'country' subfield / special field #268)
#     a record property (Order record PAID date / record property #79003)

# {
#   "queries": [
#     {
#       "target": {
#         "record": {"type": "bib"},
#         "field": {"tag": "t"
#         }
#       },
#       "expr": {
#         "op": "has",
#         "operands": ["mockingbird"]
#       }
#     },
#     "and",
#     {
# 	"target": {
# 	"record": {"type": "bib"},
# 	 "field": {"marcTag": "245",
# 		   "ind1": "0",
# 		   "ind2": "1",
# 		   "subfields": "c"
#        }
#      },
#      "expr": {
# 	"op": "has",
#        "operands": ["lee"]
#      }
#     },
#     "and",
#     {
#       "target": {
#         "record": {"type": "bib"},
#         "id": 30
#       },
#       "expr": {
#         "op": "equals",
#         "operands": ["a"]
#       }
#     },
#     "and",
#     {
#       "target": {
#         "record": {"type": "bib"},
#         "specialField": 268
#       },
#       "expr": {
#         "op": "equals",
#         "operands": ["xxu"]
#       }
#     },
#     "and",
#     {
#       "target": {
#         "record": {"type": "order"},
#         "id": 79003
#       },
#       "expr": {
#         "op": "yesterday",
#         "operands": [""]
#       }
#     }
#   ]
# }				



In [12]:
# Example 6: A query targeting data stored in a barcode field
# A compound query for Item Barcode in ("23100 10850 0098","23100108500101. ","23100108500114")

# Note that Create Lists searches record data when searching for barcodes, and does not search the normalized data stored in the barcode index. Create Lists does not find a barcode unless the data is entered exactly as it appears in the record (including all spaces and punctuation that may be stored in the record's barcode field, and that are typically removed during the indexing process).

# {
#   "queries": [
#     {
#       "target": {
#         "record": {"type": "item"},
#         "field": {"tag": "b"}
#       },
#       "expr": [
#         {
#           "op": "in",
#           "operands": [
#             "23100 10850 0098",
#             "23100108500101. ",
#             "23100108500114"
#           ]
#         }
#       ]
#     }
#   ]
# }				

barcodes = [
    "23100 10850 0098",
    "23100108500101. ",
    "23100108500114"
]

query = SierraQueryBuilder()
query.start_query(record_type='item', field_tag='b') \
    .add_expression(op='in', operands=barcodes) \
    .end_query()
print(query)


{
  "queries": [
    {
      "target": {
        "record_type": "item",
        "field_tag": "b"
      },
      "expr": {
        "op": "in",
        "operands": [
          "23100 10850 0098",
          "23100108500101. ",
          "23100108500114"
        ]
      }
    }
  ]
}
